#Problem Statement:

###Automating Data Cleaning and Feature Engineering for Data Preprocessing

In data science and machine learning projects, one of the most time-consuming and error-prone tasks is preparing the data for analysis and model training. This process typically includes tasks like handling missing values, correcting data types, removing outliers, and feature scaling, all of which are critical for improving model performance. However, manually performing these steps for every dataset can be inefficient and prone to inconsistencies, especially when dealing with large datasets or multiple projects.

This program aims to automate the
1. Data loading
2. Basic initial quality check
3. Data cleaning
4. Imputing missing values
5. Standardisation of input columns


In [ ]:
#Importing required libraries

import pandas as pd
import os
from IPython.display import display
import numpy as np
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler

In [ ]:
#Load dataset in csv, xlsx or json format.

def load_data():
  path_add = input('Give the path of the file: ')
  filename, file_ext = os.path.splitext(path_add)
  if file_ext == '.csv':
    df = pd.read_csv(path_add)
  elif file_ext == '.xlsx':
    df = pd.read_excel(path_add)
  elif file_ext == '.json':
    df = pd.read_json(path_add)

  return df


##Basic quality check

1. Display a table containing 'No. of rows', 'No. of columns', 'Duplicate columns count', 'Duplicate rows count'

2. Display a table containing 'Null values count'

3. Display a table containing 'Unique values count'

4. Display a table containing 'Data-Types'

5. Display a table containing descriptive statistics

6. Return df


In [ ]:
def basic_quality_check(df):

  df1 = pd.DataFrame([[df.shape[0], df.shape[1], df.columns.duplicated().sum(), df.duplicated().sum()]],
                       columns = ['No. of rows', 'No. of columns', 'Duplicate columns count', 'Duplicate rows count'],
                       index=['Total'])
  df2 = pd.DataFrame(df.isnull().sum(), columns = ['Null values count'])
  df3 = pd.DataFrame(df.nunique(), columns = ['Unique values count'])
  df4 = pd.DataFrame(df.dtypes, columns = ['Data types'])
  df5 = pd.DataFrame(df.describe())

  dfs = {'df1': df1, 'df2': df2, 'df3': df3, 'df4': df4, 'df5': df5}
  styled_dfs = []

  for key, values in dfs.items():
    styled_df = values.style.set_properties(**{'text-align': 'center',
                                          'background-color': '#20b2aa',
                                          'border': '1px solid black'}).set_table_styles([{'selector': 'th',
                                                                                           'props': [('background-color', '#008080'),
                                                                                                      ('border', '1px solid black')]}])
    styled_dfs.append(styled_df)

  print('*****************INITIAL QUALITY REPORT ********************')
  for i in styled_dfs:
    print('')
    display(i)
    print('')

  return df

##Data cleaning

1. Correct datatypes of the columns.
  - Ask customer about the data-time column and target column

2. If date-time column is present then convert it to date-time datatype

3. Change datatype of numeric columns to numeric.
  - int columns -> int datatype
  - float columns -> float datatype

4. Check if number of unique values in a column < 10 -> change it to category type

5. Convert rest of the columns to string type.

6. Move the target column to the last in a dataframe.

7. Return df

In [ ]:
def data_cleaning(df):

  #Asking user to enter all date-time columns within the data and target column
  print('')
  date_time = str(input('Does your data has date-time column?, 1: YES or 0: No:' ))
  print('')

  if date_time == '1':
    for i,v in enumerate(df.columns):
      print(i,v, end = '/n')
    print('')
    date_time_col = int(input('Mention date-time column by their index in your data:'))
    print('')
    date_time_col_name = df.columns[date_time_col]
    print('')
    target_col = int(input('Which is your target column?, type the index from the listed columns: '))
    print('')
    target_col_name = df.columns[target_col]

  elif date_time == '0':           #If no date-time column is present.
    print('No date-time column present in the data: ')

    for i,v in enumerate(df.columns):
      print(i,v)
    print('')
    target_col = int(input('Which is your target column?, type the index from the listed columns: '))
    print('')
    target_col_name = df.columns[target_col]
    date_time_col_name = None


  #Changing datatype of date_time columns
  if date_time == '1'and date_time_col_name:
      try:
        df[date_time_col_name] = pd.to_datetime(df[date_time_col_name], errors = 'coerce')
        print(f'{date_time_col_name} converted to date_time ')
      except ValueError:
        print('')
        print('Date-time conversion failed, skipping those columns')
        print('')


  #Changing datatype to numeric cols
  for i in df.columns:
    if i != target_col_name and i != date_time_col_name:
      try:
        df[i] = pd.to_numeric(df[i], errors = 'coerce')
        print(f'{i}, converted to numeric')
        if df[i].nunique()<10:
          df[i] = df[i].astype('category')
          print(f'{i}, converted to category')
      except Exception as e:
          df[i].astype('str')           #Changing datatype to str cols
          print(f'{i}, converted to string')


  #Moving target column to the last position.
  if target_col_name != df.columns[-1]:
    cols = list(df.columns)
    cols.remove(target_col_name)  # Remove target column from its current position
    cols.append(target_col_name)  # Insert it at the second-to-last position
    df = df.reindex(columns=cols)  # Reorder the DataFrame columns
    print(f'{target_col_name} moved to last position')


  return df

##Imputing missing values

1. Total missing values per column. If NAN > 40% remove the column

2. Total missing values per row, If NAN >40% remove the row.

3. Segregate numeric cols, categorical cols, string cols and date-time cols.

4. for numeric cols:
	- If skewed , do median imputation
	- Close to normal distribution -> Do mean imputation

5. for categorical cols:
	  - Do mode imputation

6. for string cols:
	  - impute 'xxxxx'

7. for date-time cols:
	  - forward fill

8. Return df

In [ ]:
#Imputing missing values

def data_imputation(df):

  #Checking percentage of null values per column-- dropping columns if percent_null_col > 40%
  for i in df.columns:
    if df[i].isna().sum()> 0:
      percent_null_col = df[i].isna().sum()/float(len(df[i]))
      if percent_null_col > 0.4:
        df.drop(i, axis = 1, inplace = True)
        print(f'{i} column dropped')

  #Checking percentage of null values per row-- dropping rows if percent_null_row > 40%
  if df.isna().sum().sum()> 0:         #Check null values per row only when null values are present else don't check.
    percent_null_row = df.isna().sum(axis = 1)/float(len(df.columns))
    null_rows = df[percent_null_row < 0.4]
    df.drop(null_rows.index, axis = 0, inplace = True)


  #fill NAN
  for i in df.columns:
    #In numeric cols, fill NAN with median if it is skewed else with mean values
    if pd.api.types.is_numeric_dtype(df[i]):
      skewness = df[i].skew()
      if skewness >= 0.8 or skewness <= -0.8:
        df[i] = df[i].fillna(df[i].median())
      else:
        df[i] = df[i].fillna(df[i].mean())

    #In categorical cols, fill NAN with mode
    if pd.api.types.is_categorical_dtype(df[i]):
      df[i] = df[i].fillna(df[i].mode()[0])

    #In string cols, fill NAN with some random values like 'xxxxx'
    if pd.api.types.is_string_dtype(df[i]):
      df[i] = df[i].fillna('xxxxx')

    #In datetime cols, fill NAN =via forward fill method
    if pd.api.types.is_datetime64_any_dtype(df[i]):
      df[i] = df[i].fillna(method = 'ffill')

  return df

## Removing outliers and standardizing values


1. Remove outliers
    - If yes:
        - If data is skewed:
            - Remove outliers from all the numeric columns using IQR method
        - If data is normally distirbuted
            - Remove outliers using Z-score method
    - If no:
      Dont remove the outliers and proceed with original columns

2. Ask user what type of standardisation he wants?
    - Z-score standardisation
    - MinMaxScaler
    - Robust scaling
    - MaxAbs scaling

3. Return the data frame

4. Return df

In [ ]:
def feature_engineering(df):


  #Outliers removal and standardisation to be done only on input columns.
  print('')
  stand_type = input('''What type of standardisation do you want to apply on numeric columns?
    1: Z-Score Standardization ,
    2: MinMaxScaler (Normalization),
    3: Robust scaling,
    4: MaxAbs scaling:

    ''')
  print('')

  #Removing outliers from input columns
  for i in (df.columns[:-1]):
    if pd.api.types.is_numeric_dtype(df[i]):                #Removing outliers from skewed numeric columns using IQR method
      skewness = df[i].skew()
      if skewness > 0.5 or skewness < -0.5:
        Q1 = df[i].quantile(0.25)
        Q3 = df[i].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[i] >= lower_bound) & (df[i] <= upper_bound)]
      else:
        z_score = stats.zscore(df[i])                      #Removing outliers from normally distributed numeric columns using Z-score method
        df = df[(z_score > -3) & (z_score < 3)]

  #standardisation of input columns
    if stand_type == '1':
      scaler = StandardScaler()
      df[i] = scaler.fit_transform(df[[i]])

    elif stand_type == '2':
      scaler = MinMaxScaler()
      df[i] = scaler.fit_transform(df[[i]])

    elif stand_type == '3':
      scaler = RobustScaler()
      df[i] = scaler.fit_transform(df[[i]])

    elif stand_type == '4':
      scaler = MaxAbsScaler()
      df[i] = scaler.fit_transform(df[[i]])

    else:
      print('Invalid input')
      print('By default z- score standardisation applied')
      scaler = StandardScaler()
      df[i] = scaler.fit_transform(df[[i]])


  return df

##Main_func

Putting everything together

In [ ]:
def main_func():

  import warnings
  warnings.filterwarnings("ignore")

  x1 = load_data()
  x2 = basic_quality_check(x1)
  x3 = data_cleaning(x2)
  x4 = data_imputation(x3)
  x5 = feature_engineering(x4)

  return x5

In [ ]:
main_func()

Give the path of the file: /content/sample_data/california_housing_train.csv
*****************INITIAL QUALITY REPORT ********************



,No. of rows,No. of columns,Duplicate columns count,Duplicate rows count
Total,17000,9,0,0


,Null values count
longitude,0
latitude,0
housing_median_age,0
total_rooms,0
total_bedrooms,0
population,0
households,0
median_income,0
median_house_value,0


,Unique values count
longitude,827
latitude,840
housing_median_age,52
total_rooms,5533
total_bedrooms,1848
population,3683
households,1740
median_income,11175
median_house_value,3694


,Data types
longitude,float64
latitude,float64
housing_median_age,float64
total_rooms,float64
total_bedrooms,float64
population,float64
households,float64
median_income,float64
median_house_value,float64


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000,17000.000000
mean,-119.562108,35.625225,28.589353,2643.664412,539.410824,1429.573941,501.221941,3.883578,207300.912353
std,2.005166,2.137340,12.586937,2179.947071,421.499452,1147.852959,384.520841,1.908157,115983.764387
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.790000,33.930000,18.000000,1462.000000,297.000000,790.000000,282.000000,2.566375,119400.000000
50%,-118.490000,34.250000,29.000000,2127.000000,434.000000,1167.000000,409.000000,3.544600,180400.000000
75%,-118.000000,37.720000,37.000000,3151.250000,648.250000,1721.000000,605.250000,4.767000,265000.000000
max,-114.310000,41.950000,52.000000,37937.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000




Does your data has date-time column?, 1: YES or 0: No:0

No date-time column present in the data: 
0 longitude
1 latitude
2 housing_median_age
3 total_rooms
4 total_bedrooms
5 population
6 households
7 median_income
8 median_house_value

Which is your target column?, type the index from the listed columns: 7

longitude, converted to numeric
latitude, converted to numeric
housing_median_age, converted to numeric
total_rooms, converted to numeric
total_bedrooms, converted to numeric
population, converted to numeric
households, converted to numeric
median_house_value, converted to numeric
median_income moved to last position

What type of standardisation do you want to apply on numeric columns?
    1: Z-Score Standardization ,
    2: MinMaxScaler (Normalization),
    3: Robust scaling,
    4: MaxAbs scaling:

    4



,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_house_value,median_income
2,-0.921271,0.803099,0.326923,0.126649,0.165557,0.123196,0.119510,0.176592,1.6509
3,-0.921351,0.801907,0.269231,0.264028,0.320647,0.190529,0.230848,0.151247,3.1917
4,-0.921351,0.800238,0.384615,0.255761,0.310181,0.230855,0.267620,0.134968,1.9250
5,-0.921431,0.801669,0.557692,0.243975,0.224548,0.248243,0.244127,0.152483,3.3438
6,-0.921431,0.801192,0.480769,0.511346,0.647003,0.681095,0.646578,0.169792,2.6768
...,...,...,...,...,...,...,...,...,...
16995,-0.999276,0.967342,1.000000,0.389974,0.374881,0.335553,0.376915,0.229549,2.3571
16996,-0.999357,0.969964,0.692308,0.413193,0.502379,0.441731,0.474974,0.162786,2.5179
16997,-0.999598,0.997378,0.326923,0.470888,0.505233,0.460229,0.465781,0.213476,3.0313
16998,-0.999598,0.996424,0.365385,0.470009,0.525214,0.480207,0.488253,0.176798,1.9797
